In [162]:
using Enzyme

In [163]:
function log1p_series_shifted(x)

    left = sqrt(2.)/2 - 1
    right = sqrt(2.) - 1
    sum = 0

    if x < left
        sum -= log(2.) / 2
        x = 1 + 2 * x;
    elseif right < x
        sum += log(2.) / 2
        x = (x - 1) / 2
    end
  
    y = x / (2. + x)
    y2 = y*y
    sum += y
    y *= y2
    sum += y / 3
    y *= y2
    sum += y / 5
    y *= y2
    sum += y / 7
    
    2 * sum
end


log1p_series_shifted (generic function with 1 method)

In [164]:
# E2work is a voigt (size 6)
function computeDetCM1(E2work) 

    E2work[1]*(E2work[2]*E2work[3]-E2work[4]*E2work[4]) +
    E2work[6]*(E2work[5]*E2work[4]-E2work[6]*E2work[3]) +
    E2work[5]*(E2work[6]*E2work[4]-E2work[5]*E2work[2]) +
    E2work[1] + E2work[2] + E2work[3] +
    E2work[1]*E2work[2] + E2work[1]*E2work[3] +
    E2work[2]*E2work[3] - E2work[6]*E2work[6] -
    E2work[5]*E2work[5] - E2work[4]*E2work[4]
end

computeDetCM1 (generic function with 1 method)

In [165]:
function computeMatinvSym(A, detA, Ainv)
  
  # Compute A^(-1) : A-Inverse
  B = [ A[2,2]*A[3,3] - A[2,3]*A[3,2],
        A[1,1]*A[3,3] - A[1,3]*A[3,1],
        A[1,1]*A[2,2] - A[1,2]*A[2,1],
        A[1,3]*A[2,1] - A[1,1]*A[2,3],
        A[1,2]*A[2,3] - A[1,3]*A[2,2], 
        A[1,3]*A[3,2] - A[1,2]*A[3,3]
      ]

  for m in 1:6
    Ainv[m] = B[m] / (detA)
  end

end

computeMatinvSym (generic function with 1 method)

In [166]:
function computeS(E2work, Swork, lambda, mu)

      E2 = [ 
              E2work[1] E2work[6] E2work[5];
              E2work[6] E2work[2] E2work[4];
              E2work[5] E2work[4] E2work[3]
           ]
   
      C = [ 
            1+E2[1,1] E2[1,2]   E2[1,3];
            E2[1,2]   1+E2[2,2] E2[2,3];
            E2[1,3]   E2[2,3]   1+E2[3,3]
         ]

      Cinvwork = zeros(6)
      detCm1 = computeDetCM1(E2work)
      computeMatinvSym(C, detCm1+1, Cinvwork)
      
      C_inv = [ 
                Cinvwork[1] Cinvwork[6] Cinvwork[5];
                Cinvwork[6] Cinvwork[2] Cinvwork[4];
                Cinvwork[5] Cinvwork[4] Cinvwork[3]
              ]

      logJ = log1p_series_shifted(detCm1) / 2.

      indj = [1 2 3 2 1 1]
      indk = [1 2 3 3 3 2]
      Swork = zeros(6)
      for m in 1:6
            Swork[m] = lambda*logJ*Cinvwork[m]
            for n in 1:3
                  Swork[m] += mu*C_inv[indj[m], n]*E2[n, indk[m]]
            end
      end

      Swork
end


computeS (generic function with 1 method)

In [167]:
# Run an example
E = .3
nu = .3
TwoMu = E / (1 + nu)
mu = TwoMu / 2
Kbulk = E / (3*(1 - 2*nu)) # Bulk Modulus
lambda = (3*Kbulk - TwoMu) / 3
E2work = [
    0.5895232828911128,
    0.2362491738162759,
    0.9793730522395296,
    0.2190993957421843,
    0.0126503210747925,
    0.6570956167695403
]
detCm1 = computeDetCM1(E2work)

1.9620580003448598

In [168]:
Swork = zeros(6)
computeS(E2work, Swork, lambda, mu)

6-element Vector{Float64}:
  0.09804135870174864
  0.09264024123615376
  0.10429999576217747
  0.00245763855177966
 -0.000927749557766931
  0.009382774573284715

In [169]:
# deltaEwork = rand(6)
deltaEwork = [
    0.9681576729097205,
    0.7994338113484318,
    0.2755183472001872,
    0.6500440500146469,
    0.0593948875992271,
    0.6002528007029311
]


6-element Vector{Float64}:
 0.9681576729097205
 0.7994338113484318
 0.2755183472001872
 0.6500440500146469
 0.0593948875992271
 0.6002528007029311

In [170]:
deltaE = [ 
            deltaEwork[1] deltaEwork[6] deltaEwork[5];
            deltaEwork[6] deltaEwork[2] deltaEwork[4];
            deltaEwork[5] deltaEwork[4] deltaEwork[3]
         ]

3×3 Matrix{Float64}:
 0.968158   0.600253  0.0593949
 0.600253   0.799434  0.650044
 0.0593949  0.650044  0.275518

In [171]:
E2 = [ 
        E2work[1] E2work[6] E2work[5];
        E2work[6] E2work[2] E2work[4];
        E2work[5] E2work[4] E2work[3]
     ]


3×3 Matrix{Float64}:
 0.589523   0.657096  0.0126503
 0.657096   0.236249  0.219099
 0.0126503  0.219099  0.979373

In [172]:
C = [ 
      1+E2[1,1] E2[1,2]   E2[1,3];
      E2[1,2]   1+E2[2,2] E2[2,3];
      E2[1,3]   E2[2,3]   1+E2[3,3]
    ]



3×3 Matrix{Float64}:
 1.58952    0.657096  0.0126503
 0.657096   1.23625   0.219099
 0.0126503  0.219099  1.97937

In [173]:
Cinvwork = zeros(6)
computeMatinvSym(C, detCm1+1, Cinvwork)

C_inv = [ 
           Cinvwork[1] Cinvwork[6] Cinvwork[5];
           Cinvwork[6] Cinvwork[2] Cinvwork[4];
           Cinvwork[5] Cinvwork[4] Cinvwork[3] 
        ]

3×3 Matrix{Float64}:
  0.809908   -0.438163   0.0433247
 -0.438163    1.06213   -0.114769
  0.0433247  -0.114769   0.517637

In [174]:
Cinv_contract_E = 0;
for j in 1:3
    for k in 1:3
        Cinv_contract_E += C_inv[j,k]*deltaE[j,k]
    end
end
Cinv_contract_E


1.1057616252030331

In [175]:
deltaECinv = zeros(3,3)
for j in 1:3
    for k in 1:3
        deltaECinv[j, k] = 0
        for m in 1:3
            deltaECinv[j, k] += deltaE[j, m]*C_inv[m, k]
        end
    end
end
deltaECinv

3×3 Matrix{Float64}:
  0.523683  0.20652   0.00380002
  0.16403   0.511492  0.270743
 -0.224784  0.632788  0.0705873

In [176]:
logJ = log1p_series_shifted(detCm1) / 2.

0.5429421041864513

In [177]:
deltaS = zeros(3,3)
for j in 1:3
    for k in 1:3
        for m in 1:3
            deltaS[j, k] += C_inv[j, m]*deltaECinv[m, k]
        end
    end
end
 
for j in 1:3
    for k in 1:3
        deltaS[j, k] = lambda*Cinv_contract_E*C_inv[j, k] - 2 *(lambda*logJ-mu)*deltaS[j, k]
    end
end

deltaS

3×3 Matrix{Float64}:
  0.169671   -0.0851173   0.0034737
 -0.0851173   0.219554   -0.0100671
  0.0034737  -0.0100671   0.0993075